<h1> QuantumSim - Surface Codes </h1>
Author: Michel Meulen<br>
Date: 05-12-2024<br>
<br>
This chapter introduces quantum error correction (QEC) and delves into the distance three surface code. The content presented here is based on a literature review available at INSERT LINK. To execute this notebook, two libraries are essential: quantumsim, likely familiar to you, and SOMETHING, providing circuits for the nine-qubit surface code.

A strong foundation in quantum computing, particularly the concepts explored in the chapter "QuantumSim - Shor's Nine Qubit Code," is recommended. This chapter clarifies the properties of specific quantum gates crucial for understanding the surface code, as well as the two primary error types: phase-flip (Z-errors) and bit-flip (X-errors).



In [1]:
from quantumsim import Circuit, QuantumUtil
from SurfaceCode import SurfaceCode

Surface Codes - Basics
------------------------
Quantum error correction leverages multiple physical qubits to construct more resilient logical qubits. Within the surface code architecture, qubits are classified into two categories: data qubits, which store the actual quantum information, and ancillary qubits, which facilitate error detection and correction. By measuring ancillary qubits, we can identify potential errors in data qubits and apply appropriate recovery operations to maintain data integrity. This notebook delves into the fundamentals of the surface code, starting with the fundamental building blocks and concluding in a simulation of the distance-3 surface code, providing a comprehensive understanding of the surface code.

Surface Codes - Architecture
----------------------------
Surface codes function through multiple stages. Initially, data qubits are encoded and entangled with each other. A portion of the algorithm is subsequently applied to all data qubits. It is probable that errors, such as bit-flip or phase-flip errors, will be introduced into the data qubits as a result of the application of these gates.

To detect and correct these errors, a syndrome extraction process is employed. During this stage, information about the errors, known as the syndrome, is extracted from the data qubits and stored in ancillary qubits. These ancillary qubits are then measured, and their measurement results are projected onto a classical system.

The classical system processes the measured syndrome values and feeds them into a decoder. The decoder analyzes the syndrome and determines the most likely error pattern that occurred. Based on this analysis, the decoder estimates the necessary recovery action, which involves applying corrective operations to the data qubits to reverse the effects of the errors.

To enhance the noise resilience of the qubits, this cycle of encoding, error detection, and correction can be repeated multiple times within a single quantum computation. However, this increased resilience comes at the cost of longer computation times which is crucial due qubits decoherence times and the speed of the decoder. Below is a visual representation of the different components and steps used to create a surface code system. The double lines indicate classical information flow, in contrast to quantum information flow which is indicated with single lines.

<img src="./pictures/qec_general/architecture_of_surface_code.drawio.png" alt="Architecture of the surface code" style="width:50%;"/>


Surface Codes - Encoding
--------------------------------

<img src="./pictures/surface_code/surface_code_encoding_example.drawio.png" alt="Surface code encoding circuit" style="width:30%;"/>


In [7]:
circuit = Circuit(4, 0, True)
d1, d2, d3, d4 = 0, 1, 2, 3

circuit.hadamard(d1)
circuit.cnot(d1, d2)
circuit.cnot(d2, d3)
circuit.cnot(d3, d4)

circuit.execute()
circuit.measure(True)


Measured state:
|1111>


'|1111>'

Surface Codes - X Stabilizer
----------------------------

<img src="./pictures/surface_code/surface_code_x_stabilizer_example.drawio.png" alt="X stabilizer" style="width:30%;"/>

In [39]:
circuit = Circuit(5, 1, True)
d1, d2, d3, d4, a1 = 0, 1, 2, 3, 4

# Encoding
circuit.hadamard(d1)
circuit.cnot(d1, d2)
circuit.cnot(d1, d3)
circuit.cnot(d1, d4)

# Introduce Error
circuit.bitflip_error(d1)

# Syndrome extraction
circuit.cnot(d1, a1)
circuit.cnot(d2, a1)
circuit.cnot(d3, a1)
circuit.cnot(d4, a1)

# Measurement of ancillary qubit
circuit.measurement(a1, 0)

circuit.execute()

# If an bit-flip error occured, the register[0] is 1, if there is no error the register[0] is 0
circuit.classicalBitRegister.print()

[1]


Surface Codes - Z Stabilizer
----------------------------
<img src="./pictures/surface_code/surface_code_z_stabilizer_example.drawio.png" alt="Z stabilizer" style="width:30%;"/>

In [1]:
circuit = Circuit(5, 1, True)
d1, d2, d3, d4, a1 = 0, 1, 2, 3, 4

# Encoding
circuit.hadamard(d1)
circuit.cnot(d1, d2)
circuit.cnot(d1, d3)
circuit.cnot(d1, d4)

# Introduce Error
circuit.phaseflip_error(d1)

# Syndrome extraction
circuit.hadamard(a1)
circuit.cnot(a1, d1)
circuit.cnot(a1, d2)
circuit.cnot(a1, d3)
circuit.cnot(a1, d4)
circuit.hadamard(a1)

circuit.print_gates_and_descriptions()

# Measurement of ancillary qubit
circuit.measurement(a1, 0)
circuit.execute()

# If an phase-flip error occured, the register[0] is 1, if there is no error the register[0] is 0
circuit.classicalBitRegister.print()

NameError: name 'Circuit' is not defined

Surface Codes - The Distance-Three Surface Code
-----------------------------------------------



<img src="./pictures/surface_code/surface_code_nine_qubits_layout.drawio.png" alt="NOT quantum logic gate" style="width:30%;"/>



In [2]:
surfaceCode = SurfaceCode()

# The order which you peform CNOT does matter!!
# Check this article: https://arxiv.org/pdf/1404.3747 page 3 fig 3

# https://github.com/shraggy/qiskit_surface_codes-1/blob/master/tutorials/1_surface_code_encoding.ipynb

surfaceCode.add_encoder_circuit()
# surfaceCode.add_bit_flip_circuit()

surfaceCode.add_upper_right()

surfaceCode.circuit.print_gates_and_descriptions()

# result = QuantumUtil.run_circuit(surfaceCode.circuit, 5)
# QuantumUtil.histogram_of_classical_states(result)

surfaceCode.circuit.execute(False)

surfaceCode.circuit.classicalBitRegister.print()

surfaceCode.circuit.get_classical_state_as_string()



H..........	Hadamard on qubit 0
*X.........	CNOT with control qubit 0 and target qubit 1
.*X........	CNOT with control qubit 1 and target qubit 2
..*X.......	CNOT with control qubit 2 and target qubit 3
...*X......	CNOT with control qubit 3 and target qubit 4
....*X.....	CNOT with control qubit 4 and target qubit 5
.....*X....	CNOT with control qubit 5 and target qubit 6
......*X...	CNOT with control qubit 6 and target qubit 7
.......*X..	CNOT with control qubit 7 and target qubit 8
..........H	Hadamard on qubit 10
...X......*	CNOT with control qubit 10 and target qubit 3
X.........*	CNOT with control qubit 10 and target qubit 0
*........X.	CNOT with control qubit 0 and target qubit 9
....X.....*	CNOT with control qubit 10 and target qubit 4
.X........*	CNOT with control qubit 10 and target qubit 1
..........H	Hadamard on qubit 10
.*.......X.	CNOT with control qubit 1 and target qubit 9
.........M.	Measurement on Qubit 9 projected on bit: 0
..........M	Measurement on Qubit 10 projected

'|11011000001>'